In [21]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.templates.state_preparations import MottonenStatePreparation
from sklearn.datasets import load_digits # import MNIST data set
from sklearn.utils import shuffle
from sklearn.preprocessing import normalize ,OneHotEncoder # import a normilisation function
from sklearn.model_selection import train_test_split
import torch
from torchvision import datasets, transforms

In [22]:
dev = qml.device("default.qubit", wires=13, shots=1000)
@qml.qnode(dev)
def qknn(f=None, test=None, k_val=None):
    #prepare_state(X)
    #qml.QubitStateVector(f, wires=list(range(0,6)))
    #qml.QubitStateVector(test, wires=list(range(6,12)))
    norm_f = np.linalg.norm(f)
    n_f = f/norm_f
    norm_t = np.linalg.norm(test)
    n_t = test/norm_t
    qml.templates.MottonenStatePreparation(n_f, wires=range(0,6))
    qml.templates.MottonenStatePreparation(n_t, wires=range(6,12))
    qml.Hadamard(wires=12) # This shall be the output
    for x in range(0, 6):
        qml.CSWAP(wires=[12, x, x+6]) #control=12, target1, target2
    qml.Hadamard(wires=12) # This shall be the output
    match = [qml.expval(qml.PauliZ(12))]
    return match

X, y = load_digits(n_class=2,return_X_y=True)
#print(X)
#data = torch.tensor(X.data, dtype = torch.int) 
# Convert numpy array to tensor 
#outputs = torch.tensor(data).flatten(0)
# Now split them
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=50, test_size=10, shuffle=True)
#total_records = 100 
#test_records = int(total_records * .2) 
#train_data = data[:total_records-test_records] 
#test_data = data[total_records-test_records:total_records]
#print(y_test)
#print(len(test_data[0]))
#print(len(train_data[0]))

In [23]:
k_values = [1, 3, 5, 7]
prob = []
wrong_prediction = 0
n_samples = 50
n_test = 10
nearest_k = [0]
for t in range(0, n_test):
    for iteration in range(0, n_samples):
        for k_val in k_values:
            out = qknn(X_train[iteration], X_test[t], k_val)
            prob.append(out)
            if np.rint(out) == 1.0:
                if ((out > 0.7) and (len(nearest_k) < k_val)):
                    nearest_k.append(out)
                    #print(y_test[t], y_train[iteration])
                    #print(out)
                if y_test[t]!= y_train[iteration]:
                    wrong_prediction +=1
                    #print("Matched!")
                    #print(y_test[iteration], y_train[iteration])
                    #print(out)
            elif np.rint(out) == 0.0 :
            
                if y_test[t]== y_train[iteration]:
                    wrong_prediction +=1
                    #print("No Match")
                    #print(y_test[iteration], y_train[iteration])
                    #print(out)
            nearest_k = [0]
                
accuracy = ((n_samples * n_test) - wrong_prediction) * 100/ (n_samples * n_test)
print(wrong_prediction)
print("Test Accuracy:  %", accuracy)
        

206
Test Accuracy:  % 58.8


In [24]:
np.rint(0.23)

0.0